### 1) Instalar dependências (opcional - rode no terminal ou em uma célula do notebook)
Em um notebook você pode executar (célula de bash):
```bash
pip install -U openai google-generativeai numpy scikit-learn python-dotenv langchain-google-genai
```
*Na prática, instale apenas as libs que vai usar no ambiente do Jupyter.*

In [ ]:
import os
import csv
import numpy as np

from typing import List, Tuple, Optional
from openai import OpenAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from pathlib import Path
from dotenv import load_dotenv

e:\01-projetos\11-work\11.34-engenharia-vetorial\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.5) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


### 2) Testar carregamento das variáveis de ambiente

In [2]:
# 2) Configuração e carregamento do .env (simplificado)
env_path = Path.cwd().joinpath('..', '..', '.env').resolve()
if env_path.exists():
    load_dotenv(env_path)
    print(f'🔎 .env carregado -> {env_path.resolve()}')
else:
    print('⚠️  .env não encontrado. Defina as variáveis de ambiente manualmente.')


# Checar chaves (rótulos simples)
print('OPENAI_API_KEY set? ->', bool(os.getenv('OPENAI_API_KEY')))
print('GOOGLE_API_KEY set? ->', bool(os.getenv('GOOGLE_API_KEY')))

🔎 .env carregado -> E:\01-projetos\11-work\11.34-engenharia-vetorial\.env
OPENAI_API_KEY set? -> True
GOOGLE_API_KEY set? -> True


### 3) Funções para gerar embeddings

- Vamos criar funções pequenas e claras para OpenAI e para Google Gemini (com fallback pra LangChain).
- Em sala de aula, foque no fluxo: obter texto, chamar API, receber vetor.

In [3]:

# OpenAI embedding wrapper
def openai_embedding(text: str, model: str = 'text-embedding-3-small', return_usage: bool = False) -> Tuple[List[float], Optional[int]]:
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    resp = client.embeddings.create(input=text, model=model)
    emb = resp.data[0].embedding
    if return_usage:
        usage = getattr(resp, 'usage', None)
        total_tokens = getattr(usage, 'total_tokens', None) if usage is not None else None
        return emb, total_tokens
    return emb, None


In [4]:

# Google embedding wrapper com fallback (retorna apenas embedding)
def google_embedding(text: str, model: str = 'models/text-embedding-004') -> List[float]:        
    emb = GoogleGenerativeAIEmbeddings(model=model).embed_query(text)
    return list(emb)


### 4) Exemplo prático: gerar embeddings e comparar similaridade

Vamos gerar embeddings para 3 frases e calcular a similaridade (cosine).

In [5]:
# Exemplo e comparação: gerar embeddings e calcular similaridade

def cosine_sim(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    return float(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))

texts = [
    'O gato é um animal doméstico',
    'O gato é um felino de estimação',
    'A programação é importante para engenheiros de software'
]

emb1 = emb2 = emb3 = None
backend = None


### Como a similaridade é calculada (cosine similarity)

A similaridade coseno (cosine similarity) mede o quão semelhantes são dois vetores de embeddings em termos de orientação no espaço vetorial, independentemente do seu comprimento. A fórmula usada no notebook é:

cosine_sim(v1, v2) = dot(v1, v2) / (||v1|| * ||v2||)

- dot(v1, v2) é o produto escalar entre os vetores.
- ||v|| é a norma L2 do vetor (sqrt(soma dos quadrados)).
- O resultado varia entre -1 e 1: valores próximos a 1 indicam alta similaridade, 0 indica ausência de relação linear, e -1 indica direção oposta.

Observações práticas:
- Mesmo sem normalizar explicitamente, dividir pelo produto das normas L2 normaliza o cálculo, tornando-o independente da magnitude absoluta dos vetores.
- A normalização L2 (v / ||v||) é frequentemente aplicada antes do armazenamento/índice de embeddings para acelerar e padronizar comparações (a comparação então se reduz ao produto escalar entre vetores normalizados).
- Modelos diferentes geram dimensões (dim) diferentes; a normalização evita que a dimensão afete a similaridade diretamente.


In [6]:

# Tentar OpenAI (pequeno) -> OpenAI (large) -> Google -> Erro amigável
openai_small_available = bool(os.getenv('OPENAI_API_KEY'))
openai_large_available = bool(os.getenv('OPENAI_API_KEY'))
google_available = bool(os.getenv('GOOGLE_API_KEY'))


In [7]:

# Função helper para rodar OpenAI e obter uso

def get_openai_embeddings(texts_list, model_name):
    embeddings = []
    total_tokens = 0
    for t in texts_list:
        emb, usage = openai_embedding(t, model=model_name, return_usage=True)
        embeddings.append(emb)
        if usage is not None:
            try:
                total_tokens += int(usage)
            except Exception:
                pass
    return embeddings, total_tokens if total_tokens else None


In [8]:

results = {}

# OpenAI small
if openai_small_available:
    try:
        emb_small, tokens_small = get_openai_embeddings(texts, 'text-embedding-3-small')
        results['openai_small'] = {'embeddings': emb_small, 'tokens': tokens_small, 'dim': len(emb_small[0])}
    except Exception as e:
        print('Falha ao gerar embeddings OpenAI small:', e)


In [9]:

# OpenAI large
if openai_large_available:
    try:
        emb_large, tokens_large = get_openai_embeddings(texts, 'text-embedding-3-large')
        results['openai_large'] = {'embeddings': emb_large, 'tokens': tokens_large, 'dim': len(emb_large[0])}
    except Exception as e:
        print('Falha ao gerar embeddings OpenAI large:', e)


In [10]:

# Google embeddings
if google_available:
    try:
        emb_google = [google_embedding(t) for t in texts]
        results['google'] = {'embeddings': emb_google, 'tokens': None, 'dim': len(emb_google[0])}
    except Exception as e:
        print('Falha ao gerar embeddings Google:', e)


In [11]:

# Mostrar resultados: dimensão, primeiros 5 valores e similaridades por backend
for k, v in results.items():
    print('\nBackend:', k)
    print('Dimensão:', v['dim'])
    print('Tokens (se disponível):', v['tokens'])
    for i, e in enumerate(v['embeddings']):
        print(f' Embedding {i+1} primeiros 5:', e[:5])
    # calcular similaridade
    sim_12 = cosine_sim(v['embeddings'][0], v['embeddings'][1])
    sim_13 = cosine_sim(v['embeddings'][0], v['embeddings'][2])
    print(f' Similaridade 1-2: {sim_12:.4f}')
    print(f' Similaridade 1-3: {sim_13:.4f}')



Backend: openai_small
Dimensão: 1536
Tokens (se disponível): 32
 Embedding 1 primeiros 5: [0.0014218955766409636, -0.0044931466691195965, -0.040188223123550415, 0.04201652854681015, 0.015540596097707748]
 Embedding 2 primeiros 5: [-0.012964113615453243, 0.009385946206748486, -0.061422210186719894, 0.0561358705163002, 0.017261510714888573]
 Embedding 3 primeiros 5: [0.019509999081492424, 0.027966845780611038, -0.003894458757713437, -0.009701134636998177, 0.04860801622271538]
 Similaridade 1-2: 0.8135
 Similaridade 1-3: 0.1421

Backend: openai_large
Dimensão: 3072
Tokens (se disponível): 32
 Embedding 1 primeiros 5: [-0.021825000643730164, 0.03996645659208298, -0.0028245302382856607, 0.0035684334579855204, 0.010693789459764957]
 Embedding 2 primeiros 5: [-0.016434069722890854, 0.02808525785803795, -0.002878795610740781, -0.02375573106110096, 0.024798443540930748]
 Embedding 3 primeiros 5: [0.00042473155190236866, 0.0036773430183529854, -0.023540019989013672, 0.039040759205818176, -0.002

### Como interpretar os resultados e o que estamos comparando

Nesta seção comparamos **três modelos** de embeddings: OpenAI (text-embedding-3-small), OpenAI (text-embedding-3-large) e Google (text-embedding-004). Para cada um comparamos:

- Dimensão (dim): número de componentes no vetor de embedding. Modelos maiores normalmente representam mais informações, mas são mais caros e custam mais em armazenamento/consulta.
- Tokens (quando disponível): número total de tokens usados nas chamadas (disponível para OpenAI via response.usage.total_tokens). Útil para estimativa de custo.
- Primeiros 5 valores do embedding: rápido “check” para ver distribuição/escala dos vetores.
- Similaridade 1-2 vs 1-3: comparamos a similaridade entre a frase 1 e 2 (semântica próxima) e entre 1 e 3 (semântica diferente). O **esperado** é que 1-2 tenha similaridade maior que 1-3.

Interpretação:
- Se a similaridade 1-2 > 1-3, o modelo está capturando corretamente semântica local entre as frases; quanto maior a diferença, maior a separação semântica observada.
- Uma similaridade muito alta entre 1 e 3 sugere que o modelo não distingue bem os dois conceitos ou que as frases compartilham termos/estruturas que influenciam a representação.
- Use a dimensão e tokens para equilibrar custo vs qualidade: modelos com maior dimensão costumam retornar maior qualidade semântica, mas com custo e latência maiores.


In [12]:
# Estimativa de custos (usa preços públicos quando possível)
import re
import requests
from math import ceil

# Valores padrão baseados em informações públicas (revisar e atualizar conforme o site oficial):
# - OpenAI text-embedding-3-small: $0.002 por 1K tokens
# - OpenAI text-embedding-3-large: $0.006 por 1K tokens
# - Google text-embedding-004: $0.0006 por 1K caracteres (estimativa; confirmar doc)
PRICING = {
    'openai_text-embedding-3-small': {'per_1k_tokens_usd': 0.002},
    'openai_text-embedding-3-large': {'per_1k_tokens_usd': 0.006},
    'google_text-embedding-004': {'per_1k_chars_usd': 0.0006}
}


### Metodologia de estimativa de custos

Explicação de como calculamos o custo estimado por execução:

- Para modelos OpenAI (text-embedding-3-small e -large) usamos *tokens* (quando retornados no campo usage.total_tokens) e aplicamos o custo por 1K tokens: custo = (tokens / 1000) * price_per_1k_tokens.
- Para modelos Google (text-embedding-004) usamos o comprimento do texto (characters) como proxy — o Google pode cobrar por caracteres ou por modelo, portanto usamos uma heurística de `per_1k_chars_usd` quando não há metadados mais específicos.
- `PRICING` é um dicionário editável que contém tarifas por 1k tokens ou por 1k caracteres. Esse dicionário pode ser atualizado manualmente com valores oficiais.
- Se `usage.total_tokens` não estiver disponível para um modelo, estimamos tokens com uma heurística simples (ex.: 1.3 tokens por palavra) quando necessário.

Limitações e recomendações:
- Os valores são estimativas e dependem de como cada fornecedor conta tokens/characters e do seu plano de tarifação específico.
- Para custos de produção, valide com a documentação de preços atualizada (links: OpenAI pricing e Google Vertex AI pricing) e/ou use as APIs oficiais de faturamento, quando disponíveis.


In [13]:
# Tenta buscar preços atualizados automaticamente

def try_fetch_openai_prices():
    """Tenta buscar preço de embeddings do site da OpenAI. Retorna None em caso de falha."""
    try:
        url = 'https://platform.openai.com/docs/guides/embeddings/pricing'
        r = requests.get(url, timeout=6)
        if r.status_code != 200:
            return None
        text = r.text
        # encontrar preço com padrão $X.XXXX / 1K
        m_small = re.search(r"text-embedding-3-small\D*\$([0-9]*\.?[0-9]+)\s*/\s*1K", text, re.IGNORECASE)
        m_large = re.search(r"text-embedding-3-large\D*\$([0-9]*\.?[0-9]+)\s*/\s*1K", text, re.IGNORECASE)
        if m_small or m_large:
            result = {}
            if m_small:
                result['openai_text-embedding-3-small'] = {'per_1k_tokens_usd': float(m_small.group(1))}
            if m_large:
                result['openai_text-embedding-3-large'] = {'per_1k_tokens_usd': float(m_large.group(1))}
            return result
    except Exception:
        return None


def try_fetch_google_prices():
    """Tenta buscar preços da Google (Vertex AI). Retorna None se falhar."""
    try:
        url = 'https://cloud.google.com/vertex-ai/generative-ai/pricing'
        r = requests.get(url, timeout=6)
        if r.status_code != 200:
            url2 = 'https://cloud.google.com/vertex-ai/pricing'
            r = requests.get(url2, timeout=6)
            if r.status_code != 200:
                return None
        text = r.text
        # Procurar padrões tipo $X.XXXX / 1K chars
        m = re.search(r"text-embedding-004\D*\$([0-9]*\.?[0-9]+)\s*/\s*1K", text, re.IGNORECASE)
        if m:
            return {'google_text-embedding-004': {'per_1k_chars_usd': float(m.group(1))}}
    except Exception:
        return None

# Tentar atualizar PRICING automaticamente
fopenai = try_fetch_openai_prices()
if fopenai:
    PRICING.update(fopenai)
    print('PRICING atualizado com valores OpenAI recuperados automaticamente')
else:
    print('Não foi possível recuperar preços da OpenAI automaticamente. Usando valores padrões no dict PRICING')

fgoogle = try_fetch_google_prices()
if fgoogle:
    PRICING.update(fgoogle)
    print('PRICING atualizado com valores Google recuperados automaticamente')
else:
    print('Não foi possível recuperar preços do Google automaticamente. Usando valores padrões no dict PRICING')



Não foi possível recuperar preços da OpenAI automaticamente. Usando valores padrões no dict PRICING
Não foi possível recuperar preços do Google automaticamente. Usando valores padrões no dict PRICING


In [14]:
# Função para estimar custo baseado em tokens ou caracteres

def estimate_tokens_from_text(text: str) -> int:
    # heurística simples: assume ~1.3 tokens por palavra
    words = text.split()
    return max(1, int(ceil(len(words) * 1.3)))


def estimate_cost(model_key: str, tokens: int = None, text: str = None, chars: int = None) -> float:
    pricing = PRICING.get(model_key)
    if pricing is None:
        raise KeyError(f'Modelo {model_key} sem preço definido no PRICING dict')

    if tokens is None and text is not None:
        tokens = estimate_tokens_from_text(text)
    if chars is None and text is not None:
        chars = len(text)

    if 'per_1k_tokens_usd' in pricing:
        if tokens is None:
            raise ValueError('tokens required for token-based pricing')
        return (tokens / 1000.0) * pricing['per_1k_tokens_usd']
    elif 'per_1k_chars_usd' in pricing:
        if chars is None:
            raise ValueError('chars required for char-based pricing')
        return (chars / 1000.0) * pricing['per_1k_chars_usd']
    else:
        raise KeyError('Modelo com tipo de preço desconhecido')

# Calcular custo estimado para os resultados existentes no notebook (results dict)
results_cost = {}
for k, v in results.items():
    model_key = None
    if k == 'openai_small':
        model_key = 'openai_text-embedding-3-small'
    elif k == 'openai_large':
        model_key = 'openai_text-embedding-3-large'
    elif k == 'google':
        model_key = 'google_text-embedding-004'

    tokens = v.get('tokens')
    estimated_cost = None
    try:
        if tokens:
            estimated_cost = estimate_cost(model_key, tokens=tokens)
        else:
            total_chars = sum(len(t) for t in texts)
            estimated_cost = estimate_cost(model_key, chars=total_chars)
    except Exception as e:
        estimated_cost = None
    results_cost[k] = estimated_cost

# Mostrar estimativas
for k, c in results_cost.items():
    print(f'Modelo: {k} — Estimativa de custo desta execução: ${c:.6f}' if c is not None else f'Modelo: {k} — custo não calculado (falta de info)')

print('\nObservação: revise os valores em PRICING e atualize para os preços públicos mais recentes quando necessário.')

Modelo: openai_small — Estimativa de custo desta execução: $0.000064
Modelo: openai_large — Estimativa de custo desta execução: $0.000192
Modelo: google — Estimativa de custo desta execução: $0.000068

Observação: revise os valores em PRICING e atualize para os preços públicos mais recentes quando necessário.


In [17]:
# 6) Gerar relatório: CSV + gráficos + instruções para exportar slides/PDF


# tentativas de import para pandas/matplotlib; se faltar, gerar CSV somente
have_pandas = True
have_matplotlib = True
try:
    import pandas as pd
except Exception:
    have_pandas = False
    pd = None
try:
    import matplotlib.pyplot as plt
except Exception:
    have_matplotlib = False
    plt = None

# Criar pasta para salvar resultados
out_dir = Path('../../data') / 'embeddings'
out_dir.mkdir(parents=True, exist_ok=True)

# Montar linhas para CSV
rows = []
for k, v in results.items():
    try:
        sim_12 = cosine_sim(v['embeddings'][0], v['embeddings'][1])
        sim_13 = cosine_sim(v['embeddings'][0], v['embeddings'][2])
    except Exception:
        sim_12 = None
        sim_13 = None
    rows.append({
        'model': k,
        'dim': v.get('dim'),
        'tokens': v.get('tokens'),
        'sim_12': sim_12,
        'sim_13': sim_13,
        'est_cost_usd': results_cost.get(k)
    })

csv_path = out_dir / 'comparative_results.csv'
with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['model', 'dim', 'tokens', 'sim_12', 'sim_13', 'est_cost_usd'])
    writer.writeheader()
    for r in rows:
        writer.writerow(r)
print(f'Saved CSV report to {csv_path.resolve()}')

# If pandas available, show DataFrame
if have_pandas:
    df = pd.DataFrame(rows)
    print('\nDataFrame summary:')
    display(df)
else:
    print('\nCSV content preview:')
    for r in rows:
        print(r)

# Tentar criar plots se matplotlib estiver disponível
if have_matplotlib and plt is not None:
    df = pd.DataFrame(rows) if have_pandas else None
    # Make arrays for plotting
    models = [r['model'] for r in rows]
    sim12 = [r['sim_12'] if r['sim_12'] is not None else 0 for r in rows]
    sim13 = [r['sim_13'] if r['sim_13'] is not None else 0 for r in rows]
    costs = [r['est_cost_usd'] if r['est_cost_usd'] is not None else 0 for r in rows]

    # Similarity bar chart
    plt.figure(figsize=(8, 4))
    x = range(len(models))
    plt.bar(x, sim12, width=0.4, label='Sim 1-2')
    plt.bar([i + 0.4 for i in x], sim13, width=0.4, label='Sim 1-3')
    plt.title('Similaridade (cosine) por modelo')
    plt.xticks([i + 0.2 for i in x], models)
    plt.ylim(0, 1)
    plt.legend()
    plt.tight_layout()
    plot1 = out_dir / 'similarity_comparison.png'
    plt.savefig(plot1)
    print(f'Saved plot to {plot1.resolve()}')
    plt.close()

    # Cost chart
    plt.figure(figsize=(6, 4))
    plt.bar(models, costs, color='orange')
    plt.title('Estimated cost per run (USD)')
    plt.ylabel('USD')
    plt.tight_layout()
    plot2 = out_dir / 'cost_comparison.png'
    plt.savefig(plot2)
    print(f'Saved plot to {plot2.resolve()}')
    plt.close()
else:
    print('\nMatplotlib or pandas not installed; plots skipped. To enable plots: pip install pandas matplotlib')

# Instruções para exportar o notebook em slides/PDF
print('\nInstruções para exportar o notebook como slides (Reveal.js) ou PDF:')
print('1) Para gerar slides HTML:')
print("   jupyter nbconvert --to slides notebooks/quick_test_classroom.ipynb --reveal-prefix 'https://cdnjs.cloudflare.com/ajax/libs/reveal.js/4.3.1/'")
print('2) Para gerar PDF (pode exigir LaTeX):')
print('   jupyter nbconvert --to pdf notebooks/quick_test_classroom.ipynb')

# Também posso gerar esses arquivos automaticamente se preferir (recomendo usar o terminal).

Saved CSV report to E:\01-projetos\11-work\11.34-engenharia-vetorial\data\embeddings\comparative_results.csv

DataFrame summary:


,model,dim,tokens,sim_12,sim_13,est_cost_usd
0,openai_small,1536,32.0,0.813467,0.142096,0.000064
1,openai_large,3072,32.0,0.848981,0.233285,0.000192
2,google,768,NaN,0.802875,0.391370,0.000068


Saved plot to E:\01-projetos\11-work\11.34-engenharia-vetorial\data\embeddings\similarity_comparison.png
Saved plot to E:\01-projetos\11-work\11.34-engenharia-vetorial\data\embeddings\cost_comparison.png

Instruções para exportar o notebook como slides (Reveal.js) ou PDF:
1) Para gerar slides HTML:
   jupyter nbconvert --to slides notebooks/quick_test_classroom.ipynb --reveal-prefix 'https://cdnjs.cloudflare.com/ajax/libs/reveal.js/4.3.1/'
2) Para gerar PDF (pode exigir LaTeX):
   jupyter nbconvert --to pdf notebooks/quick_test_classroom.ipynb
